In [16]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from matplotlib import pyplot as plt

In [3]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

In [6]:
display(train_data)

,case_id,equipment_id,completion_date,action_recommendation_id,action_recommendation_type,action_recommendation_category,equipment_area,usage_type,speed_category,load_category,floors_category,equipment_category,feedback
0,b7c775ad-4ebe-4848-9c53-37e7c5658e21,ele0000754,2018-10-03,ar00000174,art01,arc03,ga00000803,ut012,7,6,8,tp006,1
1,b177eefd-3946-4949-9699-0a91879350f9,ele0000789,2018-10-03,ar00000248,art02,arc03,ga00000269,ut005,4,6,2,tp001,1
2,fc2d568a-c53c-43ef-8871-a49ec22ab3b1,ele0001227,2018-10-03,ar00000158,art02,arc03,ga00000064,ut011,6,6,7,tp006,1
3,8e645922-1268-4c6b-ae6b-7b1605689cca,ele0001754,2018-10-03,ar00000105,art01,arc03,ga00000662,ut005,8,7,8,tp014,1
4,b44f10b1-9238-44db-8f0c-2d68e8c015a7,ele0002087,2018-10-03,ar00000148,art02,arc03,ga00001256,ut011,7,6,8,tp013,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115470,8eda4489-c34c-4e8a-b592-ad57b5fd842a,ele0029759,2020-01-30,ar00000124,art01,arc03,ga00000817,ut011,6,7,8,tp005,1
115471,0113eba6-6928-461a-b994-35a0b2eb9f4e,ele0029783,2020-01-30,ar00000291,art02,arc01,ga00001027,ut001,4,3,1,tp015,1
115472,c2eab0dc-218a-4a95-ab28-47449f42f660,ele0029785,2020-01-30,ar00000048,art01,arc02,ga00000355,ut011,2,1,4,tp002,1
115473,c2eab0dc-218a-4a95-ab28-47449f42f660,ele0029785,2020-01-30,ar00000293,art01,arc02,ga00000355,ut011,2,1,4,tp002,1


In [35]:
train_data[train_data['feedback'] == 0]
n_ids = train_data['action_recommendation_id'].nunique()
n_types = train_data['action_recommendation_type'].nunique()
n_categories = train_data['action_recommendation_category'].nunique()

print(n_ids, n_types, n_categories)

295 3 6


In [31]:
global_mean_train = train_data['feedback'].mean()
print(f'Mean of feedback in train data: {global_mean_train:.2f}')

train_data['equipment_area'].nunique()
# plt.hist(train_data['equipment_area'])
# plt.show()
for column in train_data.columns:
    print(f'{column}: {train_data[column].nunique()}')
#     train_data[column].hist()

Mean of feedback in train data: 0.88
case_id: 73945
equipment_id: 30000
completion_date: 466
action_recommendation_id: 295
action_recommendation_type: 3
action_recommendation_category: 6
equipment_area: 1285
usage_type: 13
speed_category: 8
load_category: 8
floors_category: 8
equipment_category: 20
feedback: 2


In [48]:


for category in train_data['action_recommendation_category'].unique():
    print(f'\n{category}')
    cat_ids = train_data.loc[train_data['action_recommendation_category'] == category, 'action_recommendation_id'].unique()
    not_cat_ids = train_data.loc[train_data['action_recommendation_category'] != category, 'action_recommendation_id'].unique()
    print(np.intersect1d(cat_ids, not_cat_ids))
    
for type in train_data['action_recommendation_type'].unique():
    print(f'\n{type}')
    type_ids = train_data.loc[train_data['action_recommendation_type'] == type, 'action_recommendation_id'].unique()
    not_type_ids = train_data.loc[train_data['action_recommendation_type'] != type, 'action_recommendation_id'].unique()
    print(np.intersect1d(type_ids, not_type_ids))
    
for type in train_data['action_recommendation_type'].unique():
    print(f'\n{type}')
    type_cats = train_data[train_data['action_recommendation_type'] == train_data.loc[train_data.loc[train_data['action_recommendation_type'] == type, 'action_recommendation_category'].duplicated()]
    not_type_cats = train_data[train_data['action_recommendation_type'] == train_data.loc[train_data.loc['action_recommendation_type'] != type, 'action_recommendation_category'].duplicated()]
    display(type_cats)
    intersection_idx = type_cats.isin(not_type_cats)
    display(train_data[['action_recommendation_type', 'action_recommendation_category']].loc[intersection_idx])
#     print(np.intersect1d(type_ids, not_type_ids))

SyntaxError: invalid syntax (<ipython-input-48-9592d1d1776c>, line 16)

In [9]:
def target_encode_mean(df: pd.DataFrame, column: str, min_category_size: int, global_mean) -> (pd.DataFrame, pd.Series):
    """
    Converts a categorical variable to a target encoded variable based on the means of the target variable.
    """
    # NaN cannot be used as a key so we have to convert NaNs to string
    df[column].replace(np.nan, 'NaN', inplace=True)
    mean_column = f'{column}_mean'
    counts_distinct = df[column].value_counts()
    counts = df[column].map(counts_distinct)
    mean_mappings = df.groupby(column).mean()['delivery_time']
    df[mean_column] = df[column].map(mean_mappings)
    df.loc[counts < min_category_size, mean_column] = global_mean
    return df, mean_mappings

def build_features(df: pd.DataFrame):
    categorical_features = [
        'action_recommendation_id',
        'action_recommendation_type',
        'action_recommendation_category',
        'equipment_area',
        'usage_type',
        'speed_category',
        
    ]
#     In our specific dataset, all of the columns are categorical
    categorical_columns = df.columns
    # Category must include at least this many samples. If not, then it's encoded to global mean delivery time.
    min_category_size = 100
    for feature in categorical_columns_to_use:
        df = target_encode_mean(df, feature, min_category_size, global_mean)
        
    target = df['feedback']
    features = 

101351

In [12]:
"""
Optimizes hyper-parameters for NGBoost using Optuna.
"""
import os

from sklearn.model_selection import train_test_split
import optuna
from dotenv import load_dotenv
from ngboost import NGBRegressor
from ngboost.distns import Bernoulli
from ngboost.scores import LogScore
from sklearn.tree import DecisionTreeClassifier
import click
from time import time

load_dotenv()


class NgbOptimizer:

    def __init__(self, study_name: str, X_train, y_train):
        # We use only the train set in optimization. Later we split it to train and validation in each iteration.
        self.X_train, self.y_train = X_train, y_train
        sampler = optuna.samplers.NSGAIISampler()
        pruner = optuna.pruners.SuccessiveHalvingPruner()
        self.study = optuna.create_study(storage=os.environ['OPTUNA_DB_URL'], study_name=study_name,
                                         direction='minimize', load_if_exists=True, sampler=sampler, pruner=pruner)

    def optimize(self):
        self.study.optimize(self._objective, n_trials=10000, catch=(ValueError,), gc_after_trial=True)

    def _objective(self, trial: optuna.Trial):
        print('Splitting train and validation data.')
        start_time = time()
        X_train, X_val, y_train, y_val = train_test_split(self.X_train, self.y_train)
        print(f'Took {time()-start_time:.2f} s.')
        tree_parameters = {
            'criterion': 'friedman_mse',    # No clue if better than MSE or MAE
            'splitter': 'best',
            'max_depth': trial.suggest_int('max_depth', 1, 1000),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 1000),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 1000),
            'max_features': trial.suggest_uniform('max_features', 0.1, 1.0),
            'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 10000),
            'min_impurity_decrease': 0,     # We use other parameters (depth, number of leaves etc.) for regularization
            'ccp_alpha': 0,                 # We use other parameters (depth, number of leaves etc.) for regularization
        }
        print('Initializing base learner.')
        start_time = time()
        tree_learner = DecisionTreeClassifier(**tree_parameters)
        print(f'Took {time()-start_time:.2f} s.')
        ngb_parameters = {
            'Dist': Bernoulli,
            'Score': LogScore,
            'Base': tree_learner,
            'natural_gradient': True,
            'n_estimators': 500,
            'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1),
            'minibatch_frac': trial.suggest_uniform('minibatch_frac', 1e-5, 1),
            'col_sample': trial.suggest_uniform('col_sample', 1e-5, 1),
            'tol': 0,
            'verbose_eval': 1,
        }
        trial.set_user_attr('tol', 0)
        early_stopping_rounds = trial.suggest_int('early_stopping_rounds', 1, 20)

        # We define val_loss_monitor so that it reports the validation score to the trial and prunes it if necessary.
        # Links to help understand what's happening here:
        # https://optuna.readthedocs.io/en/v1.0.0/tutorial/pruning.html
        # https://github.com/stanfordmlgroup/ngboost/blob/master/ngboost/ngboost.py
        # It's pretty gimmicky, but there doesn't seem to be a better way of doing it.
        step = -1

        def pruning_loss_monitor(manifold, y):
            nonlocal step
            step += 1
            score = manifold.total_score(y)
            trial.report(score, step)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
            return score

        print(f'Initializing NGBRegressor.')
        start_time = time()
        ngb = NGBRegressor(**ngb_parameters)
        print(f'Took {time()-start_time:.2f} s.')
        print(f'Fitting.')
        print(f'NGB parameters: {ngb_parameters}')
        print(f'Tree parameters: {tree_parameters}')
        print(f'early_stopping_rounds: {early_stopping_rounds}')
        start_time = time()
        ngb.fit(X=X_train, Y=y_train, X_val=X_val, Y_val=y_val, early_stopping_rounds=early_stopping_rounds,
                val_loss_monitor=pruning_loss_monitor)
        print(f'Took {time()-start_time:.2f} s.')
        trial.set_user_attr('n_estimators_actualized', len(ngb.base_models))
        log_score = ngb.evals_result['val']['LOGSCORE'][ngb.best_val_loss_itr]
        return log_score


@click.command()
@click.argument('study-name')
def optimize(study_name: str):
    NgbOptimizer(study_name).optimize()


if __name__ == '__main__':
    optimize()


ModuleNotFoundError: No module named 'src'